In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# Check dataset path
dataset_path = "/kaggle/input"
print("Available Datasets:")
print(os.listdir(dataset_path))


# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Available Datasets:
['diabetic-retinopathy-balanced']


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import DataLoader
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
import time


In [3]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using Device: {device}")

# 📌 Step 3: Define Dataset Paths (Update as needed)
train_dir = "/kaggle/input/diabetic-retinopathy-balanced/content/Diabetic_Balanced_Data/train"
val_dir = "/kaggle/input/diabetic-retinopathy-balanced/content/Diabetic_Balanced_Data/val"
test_dir = "/kaggle/input/diabetic-retinopathy-balanced/content/Diabetic_Balanced_Data/test"


Using Device: cuda


In [4]:
data_transform = transforms.Compose([
    transforms.Resize((128, 128)),  # Resize for efficiency
    transforms.RandomHorizontalFlip(p=0.2),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
    transforms.Normalize([0.5], [0.5])  # Normalize images
])

In [5]:
train_data = datasets.ImageFolder(root=train_dir, transform=data_transform)
val_data = datasets.ImageFolder(root=val_dir, transform=data_transform)
test_data = datasets.ImageFolder(root=test_dir, transform=data_transform)

# Print class labels
print("Class Labels:", train_data.classes)

Class Labels: ['0', '1', '2', '3', '4']


In [6]:
#Create DataLoaders (Parallel Loading with num_workers)
batch_size = 32 
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=4, pin_memory=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=4, pin_memory=True)


In [7]:
model = models.mobilenet_v2(weights=models.MobileNet_V2_Weights.DEFAULT)

# Modify classifier to match 5 classes
num_features = model.classifier[1].in_features
model.classifier[1] = nn.Linear(num_features, 5)
model = model.to(device)

print(model)  # Display model architecture


Downloading: "https://download.pytorch.org/models/mobilenet_v2-7ebf99e0.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-7ebf99e0.pth
100%|██████████| 13.6M/13.6M [00:00<00:00, 92.5MB/s]


MobileNetV2(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(
      (conv): Sequential(
        (0): Conv2dNormActivation(
          (0): Conv2d(16, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (1): BatchNorm2d(96, eps=

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=0.001, weight_decay=0.01)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.5)  # Reduce LR every 3 epochs

# Enable Automatic Mixed Precision (AMP)
scaler = torch.cuda.amp.GradScaler()

# Gradient Accumulation
accumulation_steps = 4  
num_epochs = 40  # You can increase if needed

best_loss = float('inf')  # Store best loss for saving best model

<ipython-input-8-becefbf20058>:6: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler()


In [9]:
for epoch in range(num_epochs):
    start_time = time.time()
    model.train()
    running_loss = 0.0
    optimizer.zero_grad()

    print(f"\n Epoch {epoch+1}/{num_epochs} starting...")

    for i, (images, labels) in enumerate(train_loader):
        images, labels = images.to(device), labels.to(device)

        with torch.amp.autocast(device_type='cuda'):
            outputs = model(images)
            loss = criterion(outputs, labels) / accumulation_steps  # Scale loss

        scaler.scale(loss).backward()  # Compute gradients

        # Only update weights if gradients exist
        if (i + 1) % accumulation_steps == 0 and any(p.grad is not None for p in model.parameters()):
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad()

        running_loss += loss.item()

        # Print loss every 10 batches
        #if (i + 1) % 10 == 0:
        #    avg_loss = running_loss / (i + 1)  # Compute running average loss
        #    print(f" Batch {i+1}/{len(train_loader)} - Avg Loss: {avg_loss:.4f}")

    # End of epoch - print epoch loss & time
    avg_epoch_loss = running_loss / len(train_loader)
    epoch_time = time.time() - start_time
    print(f"Epoch {epoch+1}/{num_epochs} completed - Avg Loss: {avg_epoch_loss:.4f}")
    print(f"Time taken: {epoch_time:.2f} seconds (~{epoch_time/60:.2f} minutes)")

    scheduler.step()  # Adjust learning rate after each epoch

    # Save best model if loss improves
    if avg_epoch_loss < best_loss:
        best_loss = avg_epoch_loss
        torch.save(model.state_dict(), "diabetic_retinopathy_best.pth")
        print("Best model saved!")



 Epoch 1/40 starting...
Epoch 1/40 completed - Avg Loss: 0.2756
Time taken: 88.02 seconds (~1.47 minutes)
Best model saved!

 Epoch 2/40 starting...
Epoch 2/40 completed - Avg Loss: 0.2418
Time taken: 64.70 seconds (~1.08 minutes)
Best model saved!

 Epoch 3/40 starting...
Epoch 3/40 completed - Avg Loss: 0.2199
Time taken: 64.16 seconds (~1.07 minutes)
Best model saved!

 Epoch 4/40 starting...
Epoch 4/40 completed - Avg Loss: 0.1854
Time taken: 63.98 seconds (~1.07 minutes)
Best model saved!

 Epoch 5/40 starting...
Epoch 5/40 completed - Avg Loss: 0.1704
Time taken: 65.15 seconds (~1.09 minutes)
Best model saved!

 Epoch 6/40 starting...
Epoch 6/40 completed - Avg Loss: 0.1578
Time taken: 65.04 seconds (~1.08 minutes)
Best model saved!

 Epoch 7/40 starting...
Epoch 7/40 completed - Avg Loss: 0.1369
Time taken: 64.57 seconds (~1.08 minutes)
Best model saved!

 Epoch 8/40 starting...
Epoch 8/40 completed - Avg Loss: 0.1253
Time taken: 65.72 seconds (~1.10 minutes)
Best model saved!


In [10]:
# Load the best model
model.load_state_dict(torch.load("diabetic_retinopathy_best.pth"))
model.eval()  # Set to evaluation mode
print("Best Model Loaded Successfully!")

correct = 0
total = 0

with torch.no_grad():  # No need to compute gradients during evaluation
    for images, labels in val_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)  # Get class with highest probability
        correct += (preds == labels).sum().item()
        total += labels.size(0)

val_accuracy = 100 * correct / total
print(f"Validation Accuracy: {val_accuracy:.2f}%")



Best Model Loaded Successfully!


<ipython-input-10-e37d03c1f886>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("diabetic_retinopathy_best.pth"))


Validation Accuracy: 79.09%


In [11]:
all_preds, all_labels = [], []

with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        all_preds.extend(preds.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())

# Print test accuracy & classification report
from sklearn.metrics import classification_report
print(" Test Set Classification Report:")
print(classification_report(all_labels, all_preds, target_names=train_data.classes))


 Test Set Classification Report:
              precision    recall  f1-score   support

           0       0.63      0.63      0.63      1000
           1       0.73      0.74      0.74       971
           2       0.67      0.61      0.64      1000
           3       0.94      0.97      0.96      1000
           4       0.96      0.99      0.98      1000

    accuracy                           0.79      4971
   macro avg       0.79      0.79      0.79      4971
weighted avg       0.79      0.79      0.79      4971

